# **A. Introduction  and Dataset Overview:**


1.   Dataset: MNIST Audio Dataset
2.   Goal: Classify spoken digits recordings (0 – 9) based on audio features
3.   Problem: Multi-class classification
4.   Target labels: Numbers from 0-9
5.   Link for the datase: https://huggingface.co/datasets/flexthink/audiomnist



**Importing necessary packages:**

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00


In [3]:
!pip install torchaudio librosa numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
import torch
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader

# **B. Data Loading and Cleaning:**

In [2]:
# Loading the Dataset: (Dataset is already split into Training, Testing and Validation)
from datasets import load_dataset

ds = load_dataset("flexthink/audiomnist")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/162 [00:00<?, ?B/s]

audiomnist.py:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/399M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/416M [00:00<?, ?B/s]

...

0000.parquet:   0%|          | 0.00/37.0M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28520 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/750 [00:00<?, ? examples/s]

In [5]:
# Printing a single dataset:
print(ds["train"][0])

{'file_name': '/storage/hf-datasets-cache/all/datasets/31152253818556-config-parquet-and-info-flexthink-audiomnist-315086ef/downloads/extracted/a5ef8955819b16af0fa318318c415faec8cf9fe6d8554d5835ca9333d2a0aed0/dataset/26/5_26_27.wav', 'audio': {'path': '5_26_27.wav', 'array': array([3.66210938e-04, 3.05175781e-04, 3.05175781e-04, ...,
       1.83105469e-04, 1.22070312e-04, 9.15527344e-05]), 'sampling_rate': 48000}, 'speaker_id': '26', 'age': 22, 'gender': 0, 'accent': 2, 'native_speaker': False, 'origin': 'Asia, China, Beijing', 'digit': 5}


# **C. Convert Data into Tensor Format:**

**Preprocessing Steps:**

In [7]:
# Defining Parameters for Preprocessing:
target_sample_rate = 16000
n_mels = 64
max_duration = 1

In [8]:
def preprocess_audio(example):
  # Convert to tensor:
  waveform = torch.tensor(example["audio"]["array"]).unsqueeze(0).to(torch.float32)

  # Resampling:
  if example["audio"]["sampling_rate"] != target_sample_rate:
    resampler = T.Resample(example["audio"]["sampling_rate"], target_sample_rate)
    waveform = resampler(waveform)

  # Trimming or padding:
  max_len = target_sample_rate * max_duration
  if waveform.shape[1] > max_len:
    waveform = waveform[:, :max_len]
  else:
    pad_len = max_len - waveform.shape[1]
    waveform = torch.nn.functional.pad(waveform, (0, pad_len))

    # Converting into Mel Spectrogram:
    mel_spec_transform = T.MelSpectrogram(sample_rate=target_sample_rate, n_mels=n_mels)
    mel_spec = mel_spec_transform(waveform)

    return {"mel_spec": mel_spec, "label": torch.tensor(example["digit"])}

Preprocessing done for Training, Testing and Validation Splits:

In [9]:
ds["train"] = ds["train"].map(preprocess_audio)
ds["valid"] = ds["valid"].map(preprocess_audio)
ds["test"] = ds["test"].map(preprocess_audio)

Map:   0%|          | 0/28520 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

**Convert Dataset into PyTorch Dataset Class:**

In [10]:
class AudioDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

        def __len__(self):
          return len(self.dataset)

          def __getitem__(self, idx):
            sample = self.dataset[idx]

            # Convert into Pytorch tensor (Ensuring it's a PyTorch tensor)
            mel_spec = torch.tensor(sample["mel_spec"], dtype=torch.float32)
            label = torch.tensor(sample["label"], dtype=torch.long)

            if mel_spec.dim() == 2:
               mel_spec = mel_spec.unsqueeze(0)

            return mel_spec, label # Output: [1, 64, Time], label as scalar tensor

# Create Dataset instances for train, validation, and test
train_dataset = AudioDataset(ds["train"])
valid_dataset = AudioDataset(ds["valid"])
test_dataset = AudioDataset(ds["test"])

# **D. Save Processed Data**

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# Define the save path inside Google Drive
save_path = "/content/drive/My Drive/audio_datasets/"

# Ensure the directory exists
import os
os.makedirs(save_path, exist_ok=True)

# Save the datasets
torch.save(train_dataset, save_path + "train_dataset.pt")
torch.save(valid_dataset, save_path + "valid_dataset.pt")
torch.save(test_dataset, save_path + "test_dataset.pt")

print("Datasets saved successfully in Google Drive!")

Datasets saved successfully in Google Drive!
